In [3]:
import pandas as pd

In [4]:
# Leer archivos de la colección
vocabulary = pd.read_csv('ejemploRibeiro/vocabulary.txt', sep='\t', header=None, names=['id_term', 'idf', 'term'])
document_vectors = pd.read_csv('ejemploRibeiro/documentVector.txt', sep=':', header=None, names=['id_doc', 'terms'])
queries = pd.read_csv('ejemploRibeiro/queries.txt', sep=':', header=None, names=['id_query', 'terms'])
relevants = pd.read_csv('ejemploRibeiro/relevantDocs', sep=':', header=None, names=['id_query', 'relevant_docs'])
information_needs = pd.read_csv('ejemploRibeiro/informationNeeds', sep=':', header=None, names=['id_in', 'text'])

In [5]:
document_vectors = document_vectors.dropna(subset=['terms'])
queries = queries.dropna(subset=['terms'])
relevants = relevants.dropna(subset=['relevant_docs'])
#print(document_vectors.head())
print(queries)
#print(document_vectors)
#print(relevants)


  id_query                     terms
1  Query 1            (72, 117, 191)
2  Query 2           (147, 195, 196)
3  Query 3   (55, 56, 141, 142, 147)
4  Query 4           (147, 179, 180)
5  Query 5           (147, 182, 184)


In [6]:
# Modelo Booleano
boolean_results = {}

# Convertir la lista de términos de los documentos en conjuntos
document_vectors['term_set'] = document_vectors['terms'].apply(lambda x: set(map(int, [term.strip() for term in x.strip(' ()').split(',')])))
queries['term_set'] = queries['terms'].apply(lambda x: set(map(int, [term.strip() for term in x.strip(' ()').split(',')])))

print(queries)
    
# Calcular conjuntos de respuestas para el modelo booleano
for idx, query_row in queries.iterrows():
    query_id = query_row['id_query']
    query_terms = query_row['term_set']
    boolean_results[query_id] = set()
    for idx, doc_row in document_vectors.iterrows():
        doc_id = doc_row['id_doc']
        doc_terms = doc_row['term_set']
        if query_terms.issubset(doc_terms):
            boolean_results[query_id].add(doc_id)

# Calcular conjuntos de respuestas para el modelo booleano
for idx, query_row in queries.iterrows():
    query_id = query_row['id_query']
    query_terms = query_row['term_set']
    boolean_results[query_id] = set()
    for idx, doc_row in document_vectors.iterrows():
        doc_id = doc_row['id_doc']
        doc_terms = doc_row['term_set']
        if query_terms.intersection(doc_terms):  # Cambio aquí
            boolean_results[query_id].add(doc_id)




  id_query                     terms                 term_set
1  Query 1            (72, 117, 191)           {72, 117, 191}
2  Query 2           (147, 195, 196)          {147, 196, 195}
3  Query 3   (55, 56, 141, 142, 147)  {141, 142, 147, 55, 56}
4  Query 4           (147, 179, 180)          {147, 180, 179}
5  Query 5           (147, 182, 184)          {184, 147, 182}


In [7]:
# Mostrar los resultados del modelo booleano
print("Resultados del modelo booleano:")
for query_id, result_docs in boolean_results.items():
    print(f"Consulta {query_id}: Documentos relevantes {result_docs}")

Resultados del modelo booleano:
Consulta Query 1: Documentos relevantes {'Doc 5', 'Doc 32', 'Doc 29', 'Doc 17', 'Doc 15', 'Doc 25', 'Doc 36'}
Consulta Query 2: Documentos relevantes {'Doc 32', 'Doc 4', 'Doc 9', 'Doc 38', 'Doc 18', 'Doc 21', 'Doc 37', 'Doc 19', 'Doc 29', 'Doc 14', 'Doc 10', 'Doc 6', 'Doc 8', 'Doc 1', 'Doc 36'}
Consulta Query 3: Documentos relevantes {'Doc 11', 'Doc 20', 'Doc 32', 'Doc 33', 'Doc 18', 'Doc 22', 'Doc 19', 'Doc 34', 'Doc 14', 'Doc 10', 'Doc 8', 'Doc 2'}
Consulta Query 4: Documentos relevantes {'Doc 27', 'Doc 32', 'Doc 13', 'Doc 18', 'Doc 35', 'Doc 30', 'Doc 10', 'Doc 8', 'Doc 7'}
Consulta Query 5: Documentos relevantes {'Doc 10', 'Doc 8', 'Doc 32', 'Doc 18'}


In [8]:
# Modelo Vectorial
vector_results = {}

In [9]:
# Calcular vectores de términos para documentos y consultas
term_vectors = {}
for idx, row in vocabulary.iterrows():
    term_id = row['id_term']
    term = row['term']
    term_vectors[term_id] = term

In [10]:
# Calcular vectores de términos para documentos
print(document_vectors)
document_vectors['term_vector'] = document_vectors['terms'].apply(lambda x: set(map(int, [term.strip() for term in x.strip(' ()').split(',')])))
document_vectors['term_vector'] = document_vectors['term_vector'].apply(lambda x: {term_id: 1 for term_id in x})

    id_doc                                              terms  \
1    Doc 1                   (44, 51, 96, 128, 129, 151, 195)   
2    Doc 2   (3, 10, 25, 35, 37, 44, 46, 51, 58, 59, 65, 8...   
3    Doc 3   (3, 4, 23, 24, 30, 31, 34, 35, 42, 51, 58, 81...   
4    Doc 4      (35, 51, 82, 96, 97, 113, 123, 135, 195, 196)   
5    Doc 5   (3, 4, 20, 32, 35, 51, 82, 96, 97, 98, 116, 1...   
6    Doc 6        (3, 6, 34, 35, 37, 46, 51, 89, 96, 97, 195)   
7    Doc 7   (10, 17, 35, 37, 41, 51, 82, 96, 120, 124, 15...   
8    Doc 8   (3, 35, 37, 46, 51, 82, 96, 124, 139, 147, 16...   
9    Doc 9        (4, 11, 51, 93, 96, 97, 118, 153, 195, 196)   
10  Doc 10   (1, 3, 4, 8, 35, 51, 60, 82, 96, 124, 147, 16...   
11  Doc 11   (35, 37, 51, 82, 96, 97, 102, 114, 124, 133, ...   
12  Doc 12   (3, 4, 5, 10, 28, 29, 35, 37, 51, 58, 59, 64,...   
13  Doc 13   (20, 51, 75, 82, 84, 87, 88, 96, 120, 124, 12...   
14  Doc 14    (11, 33, 51, 55, 69, 96, 97, 98, 124, 140, 196)   
15  Doc 15   (3, 4, 7, 35

In [11]:
# Calcular vectores de términos para consultas
queries['term_vector'] = queries['terms'].apply(lambda x: set(map(int, [term.strip() for term in x.strip(' ()').split(',')])))
queries['term_vector'] = queries['term_vector'].apply(lambda x: {term_id: 1 for term_id in x})

In [18]:
# Calcular similitud coseno entre vectores de consultas y documentos
for idx, query_row in queries.iterrows():
    query_id = query_row['id_query']
    query_vector = query_row['term_vector']
    vector_results[query_id] = {}
    for idx, doc_row in document_vectors.iterrows():
        doc_id = doc_row['id_doc']
        doc_vector = doc_row['term_vector']
        intersection = sum(query_vector.get(term_id, 0) * doc_vector.get(term_id, 0) for term_id in query_vector)
        query_norm = sum(val ** 2 for val in query_vector.values()) ** 0.5
        doc_norm = sum(val ** 2 for val in doc_vector.values()) ** 0.5
        cosine_similarity = intersection / (query_norm * doc_norm)
        # Agregar al vector de resultados solo si la similitud coseno es diferente de 0
        if cosine_similarity != 0:
            vector_results[query_id][doc_id] = cosine_similarity


In [19]:
# Encontrar los documentos más relevantes para cada consulta
for query_id, result_docs in vector_results.items():
    sorted_docs = sorted(result_docs.items(), key=lambda x: x[1], reverse=True)
    relevant_docs = [doc_id for doc_id, _ in sorted_docs]
    vector_results[query_id] = relevant_docs

In [20]:
# Mostrar los resultados del modelo vectorial
print("\nResultados del modelo vectorial:")
for query_id, result_docs in vector_results.items():
    print(f"Consulta {query_id}: Documentos relevantes {result_docs}")


Resultados del modelo vectorial:
Consulta Query 1: Documentos relevantes ['Doc 5', 'Doc 25', 'Doc 29', 'Doc 32', 'Doc 15', 'Doc 36']
Consulta Query 2: Documentos relevantes ['Doc 4', 'Doc 9', 'Doc 21', 'Doc 38', 'Doc 37', 'Doc 19', 'Doc 1', 'Doc 6', 'Doc 14', 'Doc 29', 'Doc 22', 'Doc 36']
Consulta Query 3: Documentos relevantes ['Doc 22', 'Doc 34', 'Doc 11', 'Doc 20', 'Doc 8', 'Doc 33', 'Doc 10', 'Doc 32', 'Doc 18', 'Doc 2']
Consulta Query 4: Documentos relevantes ['Doc 27', 'Doc 35', 'Doc 7', 'Doc 8', 'Doc 30', 'Doc 10', 'Doc 32', 'Doc 18', 'Doc 13']
Consulta Query 5: Documentos relevantes ['Doc 8', 'Doc 10', 'Doc 32', 'Doc 18']


In [21]:
# A continuacion, se realizaran queries similares teniendo en cuenta las necesidades de información

queries = pd.read_csv('ejemploRibeiro/queries2.txt', sep=':', header=None, names=['id_query', 'terms'])
queries = queries.dropna(subset=['terms'])

queries['term_set'] = queries['terms'].apply(lambda x: set(map(int, [term.strip() for term in x.strip(' ()').split(',')])))

# Calcular conjuntos de respuestas para el modelo booleano
for idx, query_row in queries.iterrows():
    query_id = query_row['id_query']
    query_terms = query_row['term_set']
    boolean_results[query_id] = set()
    for idx, doc_row in document_vectors.iterrows():
        doc_id = doc_row['id_doc']
        doc_terms = doc_row['term_set']
        if query_terms.issubset(doc_terms):
            boolean_results[query_id].add(doc_id)

# Calcular conjuntos de respuestas para el modelo booleano
for idx, query_row in queries.iterrows():
    query_id = query_row['id_query']
    query_terms = query_row['term_set']
    boolean_results[query_id] = set()
    for idx, doc_row in document_vectors.iterrows():
        doc_id = doc_row['id_doc']
        doc_terms = doc_row['term_set']
        if query_terms.intersection(doc_terms):  # Cambio aquí
            boolean_results[query_id].add(doc_id)

# Mostrar los resultados del modelo booleano
print("Resultados del modelo booleano:")
for query_id, result_docs in boolean_results.items():
    print(f"Consulta {query_id}: Documentos relevantes {result_docs}")


# Modelo Vectorial
vector_results = {}

# Calcular vectores de términos para documentos y consultas
term_vectors = {}
for idx, row in vocabulary.iterrows():
    term_id = row['id_term']
    term = row['term']
    term_vectors[term_id] = term

# Calcular vectores de términos para documentos
document_vectors['term_vector'] = document_vectors['terms'].apply(lambda x: set(map(int, [term.strip() for term in x.strip(' ()').split(',')])))
document_vectors['term_vector'] = document_vectors['term_vector'].apply(lambda x: {term_id: 1 for term_id in x})

# Calcular vectores de términos para consultas
queries['term_vector'] = queries['terms'].apply(lambda x: set(map(int, [term.strip() for term in x.strip(' ()').split(',')])))
queries['term_vector'] = queries['term_vector'].apply(lambda x: {term_id: 1 for term_id in x})

# Calcular similitud coseno entre vectores de consultas y documentos
for idx, query_row in queries.iterrows():
    query_id = query_row['id_query']
    query_vector = query_row['term_vector']
    vector_results[query_id] = {}
    for idx, doc_row in document_vectors.iterrows():
        doc_id = doc_row['id_doc']
        doc_vector = doc_row['term_vector']
        intersection = sum(query_vector.get(term_id, 0) * doc_vector.get(term_id, 0) for term_id in query_vector)
        query_norm = sum(val ** 2 for val in query_vector.values()) ** 0.5
        doc_norm = sum(val ** 2 for val in doc_vector.values()) ** 0.5
        cosine_similarity = intersection / (query_norm * doc_norm)
        # Agregar al vector de resultados solo si la similitud coseno es diferente de 0
        if cosine_similarity != 0:
            vector_results[query_id][doc_id] = cosine_similarity


# Encontrar los documentos más relevantes para cada consulta
for query_id, result_docs in vector_results.items():
    sorted_docs = sorted(result_docs.items(), key=lambda x: x[1], reverse=True)
    relevant_docs = [doc_id for doc_id, _ in sorted_docs]
    vector_results[query_id] = relevant_docs

# Mostrar los resultados del modelo vectorial
print("\nResultados del modelo vectorial:")
for query_id, result_docs in vector_results.items():
    print(f"Consulta {query_id}: Documentos relevantes {result_docs}")


Resultados del modelo booleano:
Consulta Query 1: Documentos relevantes {'Doc 5', 'Doc 32', 'Doc 29', 'Doc 15', 'Doc 25', 'Doc 36'}
Consulta Query 2: Documentos relevantes {'Doc 4', 'Doc 9', 'Doc 38', 'Doc 37', 'Doc 22', 'Doc 19', 'Doc 29', 'Doc 14', 'Doc 21', 'Doc 6', 'Doc 1', 'Doc 36'}
Consulta Query 3: Documentos relevantes {'Doc 11', 'Doc 20', 'Doc 32', 'Doc 33', 'Doc 18', 'Doc 22', 'Doc 34', 'Doc 10', 'Doc 8', 'Doc 2'}
Consulta Query 4: Documentos relevantes {'Doc 27', 'Doc 32', 'Doc 13', 'Doc 18', 'Doc 35', 'Doc 30', 'Doc 10', 'Doc 8', 'Doc 7'}
Consulta Query 5: Documentos relevantes {'Doc 10', 'Doc 8', 'Doc 32', 'Doc 18'}

Resultados del modelo vectorial:
Consulta Query 1: Documentos relevantes ['Doc 5', 'Doc 25', 'Doc 29', 'Doc 32', 'Doc 15', 'Doc 36']
Consulta Query 2: Documentos relevantes ['Doc 4', 'Doc 9', 'Doc 21', 'Doc 38', 'Doc 37', 'Doc 19', 'Doc 1', 'Doc 6', 'Doc 14', 'Doc 29', 'Doc 22', 'Doc 36']
Consulta Query 3: Documentos relevantes ['Doc 22', 'Doc 34', 'Doc 11', '